## This model constuct a networkx graph from data using the following nodes

### Nodes:
- Station

### Edges:
- Trip: bike trip between stations

In [1]:
import pandas as pd
import networkx as nx

In [2]:
trips_filename = '../data//201508_trip_data.csv'
stations_filename = '../data/201508_station_data.csv'

trips_pdf = pd.read_csv(trips_filename)
stations_pdf = pd.read_csv(stations_filename)

In [3]:
mapper = {
    'nodes' = [
        {
            'label'     : 'Station',
            'key'       : 'Station_id',
            'attributes': ['Station_id', 'name', 'lat', 'long', 'landmark']
        },
    ],

    'edges'= [
        {
            'lable'     : 'Trip',
            'from'      : 'Start Terminal',
            'to'        : 'End Terminal',
            'attributes': ['Trip ID', 'Start Date', 'End Date']
        }
    ]
}

In [4]:
def create_graph(graph_mapper, node_from, edges_from,  graph_type = 'networkx'):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-4-c8435f6e4ec0>, line 2)

In [5]:
g = create_graph(graph_mapper = mapper, 
                 node_from = trips_pdf, 
                 edges_from = stations_pdf, 
                 graph_type = 'networkx')

NameError: name 'create_graph' is not defined